In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.model_selection import train_test_split

In [2]:
channel_values = open('..\Datasets\\NO2\\NO2.txt').read().split()

In [3]:
data = np.array(channel_values)
data = data.reshape(-1, 8)
data.shape

(500, 8)

In [4]:
df = pd.DataFrame(data)
df

,0,1,2,3,4,5,6,7
0,3.71844,7.6912,9.2,4.8,-0.1,74.4,20,600
1,3.10009,7.69894,6.4,3.5,-0.3,56,14,196
2,3.31419,4.81218,-3.7,0.9,-0.1,281.3,4,513
3,4.38826,6.95177,-7.2,1.7,1.2,74,23,143
4,4.3464,7.51806,-1.3,2.6,-0.1,65,11,115
...,...,...,...,...,...,...,...,...
495,4.30946,7.68202,3.5,5,-1,78,11,166
496,2.94444,6.52942,9.5,6.5,-0.8,210,10,35
497,4.17439,7.75791,5.2,4.6,-0.8,214,14,176
498,2.95491,5.78996,8.4,0.5,-2.6,108.5,7,588


In [5]:
df = df.dropna()
df

,0,1,2,3,4,5,6,7
0,3.71844,7.6912,9.2,4.8,-0.1,74.4,20,600
1,3.10009,7.69894,6.4,3.5,-0.3,56,14,196
2,3.31419,4.81218,-3.7,0.9,-0.1,281.3,4,513
3,4.38826,6.95177,-7.2,1.7,1.2,74,23,143
4,4.3464,7.51806,-1.3,2.6,-0.1,65,11,115
...,...,...,...,...,...,...,...,...
495,4.30946,7.68202,3.5,5,-1,78,11,166
496,2.94444,6.52942,9.5,6.5,-0.8,210,10,35
497,4.17439,7.75791,5.2,4.6,-0.8,214,14,176
498,2.95491,5.78996,8.4,0.5,-2.6,108.5,7,588


In [6]:
df_subset = df[df.columns[1:]]
df_subset

,1,2,3,4,5,6,7
0,7.6912,9.2,4.8,-0.1,74.4,20,600
1,7.69894,6.4,3.5,-0.3,56,14,196
2,4.81218,-3.7,0.9,-0.1,281.3,4,513
3,6.95177,-7.2,1.7,1.2,74,23,143
4,7.51806,-1.3,2.6,-0.1,65,11,115
...,...,...,...,...,...,...,...
495,7.68202,3.5,5,-1,78,11,166
496,6.52942,9.5,6.5,-0.8,210,10,35
497,7.75791,5.2,4.6,-0.8,214,14,176
498,5.78996,8.4,0.5,-2.6,108.5,7,588


In [7]:
label = df[df.columns[0]]
label

0      3.71844
1      3.10009
2      3.31419
3      4.38826
4       4.3464
        ...   
495    4.30946
496    2.94444
497    4.17439
498    2.95491
499    4.03247
Name: 0, Length: 500, dtype: object

In [8]:
data = df_subset
labels_true = label
scaler = StandardScaler()
random_seed = np.load(file="..\Seeds\seed1.npy")

for i in range(20):
    X_train, X_test, y_train, y_test = train_test_split(data, labels_true, test_size=0.2, random_state=random_seed[i])
    X_train = scaler.fit_transform(X_train.values.astype(np.float32))
    X_test = scaler.transform(X_test.values.astype(np.float32))
    
    np.save("..\datasets\\NO2\\X_train" + str(i) +".npy", X_train)
    np.save("..\datasets\\NO2\\X_test" + str(i) +".npy", X_test)
    np.save("..\datasets\\NO2\\y_train" + str(i) +".npy", y_train)
    np.save("..\datasets\\NO2\\y_test" + str(i) +".npy", y_test)

In [9]:
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import pandas as pd

total_test_r2 = []
total_test_mse = []

iteration = 20

seed_rf = np.load(file="..\Seeds\seed2.npy")

for iters in range(iteration):
    print("Iteration:", iters)
    name1 = "..\datasets\\NO2\X_train" + str(iters) + ".npy"
    name2 = "..\datasets\\NO2\X_test" + str(iters) + ".npy"
    name3 = "..\datasets\\NO2\y_train" + str(iters) + ".npy"
    name4 = "..\datasets\\NO2\y_test" + str(iters) + ".npy"
    
    X_train = np.load(name1, allow_pickle=True).astype(np.float32)
    print(X_train.shape)
    X_test = np.load(name2, allow_pickle=True).astype(np.float32)
    y_train = np.load(name3, allow_pickle=True).astype(np.float32).reshape(-1, 1)
    y_test = np.load(name4, allow_pickle=True).astype(np.float32).reshape(-1, 1)
    
    estimator=RandomForestRegressor(n_estimators=100, random_state=seed_rf[iters])
    estimator.fit(X_train, y_train.ravel())
    prediction_=estimator.predict(X_test)
    
    total_test_r2.append(r2_score(y_test, prediction_))
    total_test_mse.append(mean_squared_error(y_test, prediction_))

Iteration: 0
(400, 7)
Iteration: 1
(400, 7)
Iteration: 2
(400, 7)
Iteration: 3
(400, 7)
Iteration: 4
(400, 7)
Iteration: 5
(400, 7)
Iteration: 6
(400, 7)
Iteration: 7
(400, 7)
Iteration: 8
(400, 7)
Iteration: 9
(400, 7)
Iteration: 10
(400, 7)
Iteration: 11
(400, 7)
Iteration: 12
(400, 7)
Iteration: 13
(400, 7)
Iteration: 14
(400, 7)
Iteration: 15
(400, 7)
Iteration: 16
(400, 7)
Iteration: 17
(400, 7)
Iteration: 18
(400, 7)
Iteration: 19
(400, 7)


In [10]:
print("Median R2:", np.mean(np.array(total_test_r2)))
print("Median RMSE:", np.mean(np.sqrt(np.array(total_test_mse))))

Median R2: 0.6041927190919582
Median RMSE: 0.45446315899520845
